<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [3]:
#===============================================
# Basic Imports
BASE_FOLDER = '../../'
%run -i ..\..\utility\feature_extractor\JupyterLoad_feature_extractor.py
%run -i ..\..\utility\modeling\JupyterLoad_modeling.py

path = BASE_FOLDER + '/dataset/extdia_v1/pump6dB00_EDiaV1HPaug0_pandaDisc.pkl'
tt_split(path)
with open(path, 'rb') as f:
    df = pickle.load(f)

load feature_extractor_mother
load feature_extractor_mel_spectra
load feature_extractor_psd
load feature_extractor_ICA2
load feature_extractore_pre_nnFilterDenoise
load extractor_diagram_mother
load Simple_FIR_HP
load TimeSliceAppendActivation
load load_data
Load split_data
Load anomaly_detection_models
Load pseudo_supervised_models
Load detection_pipe
../..//dataset/extdia_v1/pump6dB00_EDiaV1HPaug0_pandaDisc.pkl --> Done


In [4]:
df.columns

Index(['path', 'abnormal', 'datafile_idx', 'MEL_denbssm', 'PSD_denbssm',
       'MEL_bssm', 'PSD_bssm', 'MEL_raw', 'PSD_raw', 'MEL_den', 'PSD_den',
       'ICA_demix', 'SNR', 'machine', 'ID', 'train_set'],
      dtype='object')

In [5]:
diagrams = ['extdia_v1']
machines = ['pump'#, 'fan', 'slider', 'valve'
            ]
SNRs = ['6dB'#, 'min6dB'
        ]
IDs = ['00'#, '02', '04', '06'
        ]
features = ['PSD_den'#, 'PSD_den'
            ]

tasks = [{
        'path_descr':path, 
        'feat':{'function':'frame', 'frames':5}, 
        'feat_col':feature, 
        'SNR':SNR, 
        'machine':machine, 
        'ID':ID,
        'BASE_FOLDER':BASE_FOLDER} 
        for diagram in diagrams
        for machine in machines
        for SNR in SNRs
        for ID in IDs
        for feature in features
        ]

In [28]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, FastICA

preprocessing = [
    (FastICA, {'n_components':40, 'algorithm':'parallel'}),
    (StandardScaler, {})
]

#modeling = (uni_LogisticRegression, {})
modeling = (uni_RandomForestClassifier, {})
pipes = [Pipe(preprocessing, modeling) for i in range(len(tasks))]

In [29]:
pipes[0].run_pipe(tasks[0])

...loading data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.9059761484612532
pipe saved to pickle


True

In [27]:
pipes[0].df_test.pred_labels.value_counts()

 1.0    77639
-1.0    10735
Name: pred_labels, dtype: int64

In [21]:
ground_truth = pipes[0].df_train.abnormal.replace(to_replace=1, value=-1).replace(to_replace=0, value=1)
ground_truth

0         1.0
1         1.0
2         1.0
3         1.0
4         1.0
         ... 
533329   -1.0
533330   -1.0
533331   -1.0
533332   -1.0
533333   -1.0
Name: abnormal, Length: 533334, dtype: float64

In [22]:
pipes[0].evaluate(data_test, ground_truth)
#pipes[0].df_test.pred_labels

ValueError: X has 400 features per sample; expecting 40

In [8]:
   
dd = pickle.load( open( r'.\pipes\MEL_den_frame5_6dB_pump_ID00_20200504_133517.pkl', "rb" ))

In [15]:
dd.roc_auc
dd.model_args


{}

# ignore part below unless multithreading is needed

In [5]:
#lw = LoggerWrap()

# create the threads
n_jobs = 4
worker_list = []
queue = Queue()
for worker in range(n_jobs):
    worker = PipeThread(queue)
    worker.daemon = True
    worker.start()
    worker_list.append(worker)

In [5]:
# fill the Queue
#lw.log('multithread mode filling the queue' )
for pipe, task in (zip(pipes, tasks)):
    queue.put((pipe, task))

...loading data
data loading completed

...preprocessing data


In [7]:
for worker in worker_list:
    worker.stop = True